In [75]:
from geoband.API import *
from geopandas.tools import sjoin


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

import pydeck as pdk
import shapefile
import os
import re
import matplotlib 
matplotlib.font_manager._rebuild()
import seaborn as sns

import mapboxgl
from mapboxgl.viz import *
from mapboxgl.utils import create_color_stops

from matplotlib import font_manager, rc
import platform

import requests; from urllib.parse import urlparse
import warnings
warnings.filterwarnings("ignore")

from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import pandas as pd
import json
from pandas import json_normalize
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from geoband.API import *
import geopandas as gpd
import warnings
warnings.filterwarnings("ignore")
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from geopandas import GeoDataFrame
# GetCompasData로 주피터 노트북 경로에 파일 저장
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
from pandas import json_normalize

## 공공데이터 포털
- 병원 / 의원 정보 수집

In [1]:
from geoband.API import *
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import geopandas as gpd
import json
from pandas import json_normalize
import folium
from folium.plugins import MarkerCluster
import matplotlib.pyplot as plt
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
from folium import Choropleth
import matplotlib
from datetime import datetime 
# GetCompasData로 주피터 노트북 경로에 파일 저장
from shapely.geometry import Point, Polygon, LineString
from fiona.crs import from_string
matplotlib.rcParams['font.family']="Malgun Gothic"
import requests
from bs4 import BeautifulSoup
import glob, os
import urllib
from urllib.parse import urlencode, quote_plus
import requests
from bs4 import BeautifulSoup
from pandas import json_normalize
import xml.dom.minidom

### 약국주소 가져오기

In [2]:
def 약국매핑(지역):
    ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
    open_url = 'http://apis.data.go.kr/B552657/ErmctInsttInfoInqireService/getParmacyListInfoInqire'

    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                    quote_plus('Q0') : 지역,
                                   quote_plus('pageNo') : '1',
                                   quote_plus('numOfRows') : '10000' })
    open_url_get = open_url + queryParams

    res = requests.get(open_url_get)
    datas = BeautifulSoup(res.content,"html.parser").find_all("item")
    기관주소 = [] ; 기관명 = [] ; 경도 = [] ; 위도 = []

    for data in datas:
        기관주소.append(data.find("dutyaddr").get_text())
        기관명.append(data.find("dutyname").get_text())
        try:
            경도.append(data.find("wgs84lon").get_text()) 
        except: 경도.append(0)
        try:
            위도.append(data.find("wgs84lat").get_text()) 
        except: 위도.append(0)

    df = pd.DataFrame(columns = ["기관주소","기관명","경도","위도"])

    df["기관주소"] = 기관주소
    df["기관명"] = 기관명
    df["경도"] = 경도
    df["위도"] = 위도
    return df

In [3]:
약국_수원 = 약국매핑('수원')
약국_진주 = 약국매핑('진주시')
약국_창원 = 약국매핑('창원시')

서울 약국 가져오기

In [4]:
약국_서울 = pd.read_csv("서울특별시_약국_정보.csv",encoding="cp949")[["주소","약국명","병원경도","병원위도"]]
약국_서울.columns = ["기관주소","기관명","경도","위도"]

In [5]:
약국_4개시 = pd.concat([약국_수원,약국_서울,약국_진주,약국_창원]).reset_index(drop=True)
약국_4개시.columns = ["기관주소","기관명","lon","lat"]
약국_4개시 = 약국_4개시[약국_4개시["lat"].notnull()]

In [6]:
약국_4개시.to_csv("4개시도_약국.csv",encoding="utf-8-sig")

### 병원 주소 가져오기

In [7]:
def 병원매핑(지역):
    ServiceKey = "/Vg1J8MmKzuTqcg1RFWfc3U7B6OLQGu28VZvd9EkkL7l0+IYsdB5Oe0eTm85cUMsNyjZVrFo7GC8fKNxLqKRDA=="
    open_url = 'http://apis.data.go.kr/B552657/HsptlAsembySearchService/getHsptlMdcncListInfoInqire'

    queryParams = '?' + urlencode({ quote_plus('ServiceKey') : ServiceKey,
                                    quote_plus('Q0') : 지역,
                                   quote_plus('pageNo') : '1',
                                   quote_plus('numOfRows') : '10000' })
    open_url_get = open_url + queryParams

    res = requests.get(open_url_get)
    datas = BeautifulSoup(res.content,"html.parser").find_all("item")
    기관주소 = [] ; 기관명 = [] ; 경도 = [] ; 위도 = []

    for data in datas:
        기관주소.append(data.find("dutyaddr").get_text())
        기관명.append(data.find("dutyname").get_text())
        try:
            경도.append(data.find("wgs84lon").get_text()) 
        except: 경도.append(0)
        try:
            위도.append(data.find("wgs84lat").get_text()) 
        except: 위도.append(0)

    df = pd.DataFrame(columns = ["기관주소","기관명","경도","위도"])

    df["기관주소"] = 기관주소
    df["기관명"] = 기관명
    df["경도"] = 경도
    df["위도"] = 위도
    return df

In [10]:
병원_수원 = 병원매핑('수원')
#병원_서울 = 병원매핑('서울특별시')
병원_진주 = 병원매핑('진주시')
병원_창원 = 병원매핑('창원시')

In [13]:
병원_서울 = pd.read_csv("서울특별시_병의원_정보.csv",encoding="cp949")[["주소","기관명","병원경도","병원위도"]]
병원_서울.columns = ["기관주소","기관명","경도","위도"]

In [14]:
병원_4개시 = pd.concat([병원_수원,병원_서울,병원_진주,병원_창원])
병원_4개시.columns = ["기관주소","기관명","lon","lat"]
병원_4개시 = 병원_4개시[병원_4개시["lat"].notnull()]

In [15]:
병원_4개시.to_csv("4개시도_병원.csv",encoding="utf-8-sig")

### 전국 지하철역 데이터

In [55]:
subway = pd.read_csv("전국도시철도역사정보표준데이터.csv",encoding="cp949")[["역사명","역사도로명주소","역경도","역위도"]]
subway.columns = ["기관명","기관주소","lon","lat"]

subway = subway[(subway["기관주소"].str.contains("서울")) | (subway["기관주소"].str.contains("수원"))]
# 서울 수원 데이터만 추출

others_subway = subway.drop_duplicates(["기관주소"],keep="first").reset_index(drop=True)

In [56]:
others_subway.to_csv("4개시도_지하철역.csv",encoding="utf-8-sig")

### 전국 초중고 데이터

In [69]:
others_school = pd.read_csv("한국교원대학교_초중등학교위치.csv",encoding="cp949")[["학교명","학교급구분","소재지도로명주소","경도","위도"]]
others_school = others_school[(others_school["소재지도로명주소"].str.contains("창원")) 
                              | (others_school["소재지도로명주소"].str.contains("진주"))
                              | (others_school["소재지도로명주소"].str.contains("서울"))
                              | (others_school["소재지도로명주소"].str.contains("수원"))]

others_school.columns = ["학교명","학교급구분","소재지도로명주소","lon","lat"]

In [70]:
others_school.to_csv("4개시도_초중고.csv",encoding="utf-8-sig")

### 전국 어린이집

In [88]:
어린이집_창원 = pd.read_excel("어린이집기본정보조회_창원.xls")
어린이집_수원 = pd.read_excel("어린이집기본정보조회_수원.xls")
어린이집_진주 = pd.read_excel("어린이집기본정보조회_진주.xls")
어린이집_서울 = pd.read_excel("어린이집기본정보조회_서울.xls")

others_baby = pd.concat([어린이집_창원,어린이집_수원,어린이집_진주,어린이집_서울])[["어린이집명","운영현황","주소","경도","위도"]]

others_baby = others_baby[(others_baby["운영현황"] == "정상") | (others_baby["운영현황"] == "재개")].reset_index(drop=True)
others_baby.columns = ["어린이집명","운영현황","주소","lon","lat"]
# 정상 운영되는 곳만

In [89]:
others_baby.to_csv("4개시도_어린이집.csv",encoding="utf-8-sig")